<a href="https://colab.research.google.com/github/ilya-lykov/google_colab_labs/blob/main/lab_spark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=558367e6d1b33864b938d9015a43391aa098454bc2c0a5b89cf2cf98abaf035b
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Новый раздел

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
spark = (SparkSession.builder.appName("Test").getOrCreate())

In [5]:
acc = spark.read.csv("/content/drive/MyDrive/University/data/2015/DfTRoadSafety_Accidents_2015.csv", header=True, inferSchema=True)

In [6]:
acc.show()

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-------------------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+
|Accident_Index|Location_Easting_OSGR|Location_Northing_OSGR|Longitude| Latitude|Police_Force|Accident_Severity|Number_of_Vehicles|Number_of_Casualties|      Date|Day_of_Week|               Time|Local_Authority_(District)|Local_Authority_(Highway)|1st_Road_Class|1st_Road_Number|Road_Type|Speed_limit|Junction_Detail|Junction_Control|2nd_Road_Class|2nd_Road_Numbe

In [8]:
acc1 = acc.select("Accident_Index", "Accident_Severity", "Weather_Conditions")

In [9]:
acc1.show()

+--------------+-----------------+------------------+
|Accident_Index|Accident_Severity|Weather_Conditions|
+--------------+-----------------+------------------+
| 201501BS70001|                3|                 1|
| 201501BS70002|                3|                 1|
| 201501BS70004|                3|                 2|
| 201501BS70005|                3|                 1|
| 201501BS70008|                2|                 2|
| 201501BS70009|                3|                 1|
| 201501BS70010|                3|                 8|
| 201501BS70011|                3|                 1|
| 201501BS70012|                3|                 1|
| 201501BS70013|                3|                 8|
| 201501BS70014|                3|                 2|
| 201501BS70015|                3|                 1|
| 201501BS70016|                3|                 1|
| 201501BS70017|                3|                 1|
| 201501BS70018|                3|                 1|
| 201501BS70019|            

In [25]:
acc2 = acc1.filter(F.col("Accident_Severity")>1)

print(f"Количество записей в acc1: {acc1.count()}")
print(f"Количество записей в acc2: {acc2.count()}")

# acc3 = acc1.filter(F.col("Accident_severity)")>2 | (F.col("Weather_Conditions")>1))
# acc4 = acc1.filter(F.col("Accident_severity)")>2 & (F.col("Weather_Conditions")>1))
# acc5 = acc1.filter(F.col("Accident_severity)")>2 & ~(F.col("Weather_Conditions")>1))

acc6 = acc1.groupBy(F.col("Weather_Conditions")).agg(F.count("*").alias("count_by_weather"))
acc6.show()



Количество записей в acc1: 140086
Количество записей в acc2: 138468
+------------------+----------------+
|Weather_Conditions|count_by_weather|
+------------------+----------------+
|                 1|          113968|
|                 6|             175|
|                 3|             540|
|                 5|            2366|
|                 9|            2470|
|                 4|            2355|
|                 8|            1986|
|                 7|             662|
|                 2|           15564|
+------------------+----------------+



In [26]:

acc7 = (acc1
        .groupBy(F.col("Weather_Conditions"))
        .agg(F.count("*").alias("count_by_weather"))
        .orderBy(F.col("count_by_weather").desc())
        )

acc7.show()



+------------------+----------------+
|Weather_Conditions|count_by_weather|
+------------------+----------------+
|                 1|          113968|
|                 2|           15564|
|                 9|            2470|
|                 5|            2366|
|                 4|            2355|
|                 8|            1986|
|                 7|             662|
|                 3|             540|
|                 6|             175|
+------------------+----------------+



In [27]:
acc8 = (acc1
        .groupBy(F.col("Accident_Severity"), F.col("Weather_Conditions"))
        .agg(F.count("*").alias("count_by_group"))
        .orderBy(F.col("Accident_Severity").asc(), F.col("Weather_Conditions").desc())
        )
acc8.show()



+-----------------+------------------+--------------+
|Accident_Severity|Weather_Conditions|count_by_group|
+-----------------+------------------+--------------+
|                1|                 9|            32|
|                1|                 8|            13|
|                1|                 7|            13|
|                1|                 5|            28|
|                1|                 4|            39|
|                1|                 3|             2|
|                1|                 2|           165|
|                1|                 1|          1326|
|                2|                 9|           253|
|                2|                 8|           247|
|                2|                 7|           124|
|                2|                 6|            22|
|                2|                 5|           353|
|                2|                 4|           392|
|                2|                 3|            67|
|                2|         

In [28]:
acc_origin_2 = acc.select("Accident_Index", "Accident_Severity", "Weather_Conditions", "Date", "Day_of_week")

acc9 = (acc_origin_2
        .groupBy(F.col("Accident_Severity"), F.col("Weather_Conditions"))
        .agg(F.count("*").alias("count_by_group"),  F.min(F.col("Date").alias("min_date")), F.avg(F.col("Day_of_week")))
        .orderBy(F.col("Accident_Severity").asc(), F.col("Weather_Conditions").desc())
        )
acc9.show()


+-----------------+------------------+--------------+---------------------+------------------+
|Accident_Severity|Weather_Conditions|count_by_group|min(Date AS min_date)|  avg(Day_of_week)|
+-----------------+------------------+--------------+---------------------+------------------+
|                1|                 9|            32|           01/11/2015|           4.15625|
|                1|                 8|            13|           04/04/2015|4.3076923076923075|
|                1|                 7|            13|           01/10/2015|2.8461538461538463|
|                1|                 5|            28|           01/01/2015| 4.071428571428571|
|                1|                 4|            39|           01/01/2015| 4.538461538461538|
|                1|                 3|             2|           02/03/2015|               3.0|
|                1|                 2|           165|           01/01/2015|4.1878787878787875|
|                1|                 1|          13

In [29]:
veh = spark.read.csv("/content/drive/MyDrive/University/data/2015/DfTRoadSafety_Vehicles_2015.csv", header=True, inferSchema=True)

veh.show()

df = acc.join(veh, ['Accident_Index'], 'inner')

+--------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+-----------------+---------------------+
|Accident_Index|Vehicle_Reference|Vehicle_Type|Towing_and_Articulation|Vehicle_Manoeuvre|Vehicle_Location-Restricted_Lane|Junction_Location|Skidding_and_Overturning|Hit_Object_in_Carriageway|Vehicle_Leaving_Carriageway|Hit_Object_off_Carriageway|1st_Point_of_Impact|Was_Vehicle_Left_Hand_Drive?|Journey_Purpose_of_Driver|Sex_of_Driver|Age_of_Driver|Age_Band_of_Driver|Engine_Capacity_(CC)|Propulsion_Code|Age_of_Vehicle|Driver_IMD_Decile|Driver_Home_Area_Type|
+--------------+-----------------+------------+-----------------------+-------

In [30]:
df = acc.join(veh, ['Accident_Index'], 'inner')

df.show()

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-------------------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+------------

In [ ]:
df1 = spark.createDataFrame([(1, 'a'),(2, 'b'), (3,'c'),(4,'d')], ["id", "value"] )
df2 = spark.createDataFrame([(2, 'x'),(3, 'y'), (4,'z')], ["id", "value2"] )
df1.show()
df2.show()

# df1.join(df2, ["id"], 'inner').show()
# df1.join(df2, ["id"], 'left').show()
# df1.join(df2, ["id"], 'right').show()
# df1.join(df2, ["id"], 'left_anti').show()
# df1.join(df2, ["id"], 'outer').show()
(df1.withColumnRenamed("id", "id_1").join(df2.withColumnRenamed("id", "id_2"), how = 'cross')
.orderBy("id_1", "id_2").show())





# df3.show()


+---+-----+
| id|value|
+---+-----+
|  1|    a|
|  2|    b|
|  3|    c|
|  4|    d|
+---+-----+

+---+------+
| id|value2|
+---+------+
|  2|     x|
|  3|     y|
|  4|     z|
+---+------+

+----+-----+----+------+
|id_1|value|id_2|value2|
+----+-----+----+------+
|   1|    a|   2|     x|
|   1|    a|   3|     y|
|   1|    a|   4|     z|
|   2|    b|   2|     x|
|   2|    b|   3|     y|
|   2|    b|   4|     z|
|   3|    c|   2|     x|
|   3|    c|   3|     y|
|   3|    c|   4|     z|
|   4|    d|   2|     x|
|   4|    d|   3|     y|
|   4|    d|   4|     z|
+----+-----+----+------+

